In [3]:
import pandas as pd
import numpy as np
import functions as fc
import imlconnect
from datetime import datetime, timedelta

In [4]:
print("Ce script sert à identifier les dossiers d'analyses qui doivent être clos par Hypervision")
print("DA concernés : 30,64,25,38,23,24,39,28,42,27,54,22,61,26,59,29,56")
print("Motifs de nettoyage:  DA non clos alors que C remplacé, DA non clos alors que C déposé, DA Défaillance Mat non clos alors que C a recollecté, DA Défaillance Mat non clos alors que C coupé au CCPI")

print("Entrez la date du jour svp ?\n")
j = input("Jour ? (format numérique svp!)\n")
while len(j)!=2:
    j = input("Jour ? (2 caractères et format numérique svp!)\n")
while True:
    try:
        val = int(str(j))
        while val<1 or val>31:
            j = input("Jour ? Entre 01 et 31 svp!)\n") 
            val = int(str(j))
    except ValueError:
        # Not a valid number
        print("Ce n'est pas un nombre entier\n")
        j = input("Jour ? (format numérique svp!)\n")
    else:
        print("Succès !\n")
        # No error; stop the loop
        break  

m = input("Mois ? (format numérique svp!)\n")
while len(m)!=2:
    m = input("Mois ? (2 caractères et format numérique svp!)\n")
while True:
    try:
        val1 = int(str(m))
        while val1<1 or val1>12:
            m = input("Mois ? Entre 01 et 12 svp!)\n") 
            val1 = int(str(m))
    except ValueError:
        # Not a valid number
        print("Ce n'est pas un nombre entier\n")
        m = input("Mois ? (format numérique svp!)\n")
    else:
        print("Succès !\n")# No error; stop the loop
        break

a = input("Année en cours? (format numérique svp!)\n")
while len(a)!=4:
    a = input("Année ? (4 caractères et format numérique svp!)\n")
while True:
    try:
        val2 = int(str(a))
        while val2!=datetime.now().year:
            a = input("Année doit être année en cours svp!)\n") 
            val2 = int(str(a))
    except ValueError:
        # Not a valid number
        print("Ce n'est pas un nombre entier\n")
        a = input("Année ? (format numérique svp!)\n")
    else:
        print("Succès !\n")# No error; stop the loop
        break

Ce script sert à identifier les dossiers d'analyses qui doivent être clos par Hypervision
DA concernés : 30,64,25,38,23,24,39,28,42,27,54,22,61,26,59,29,56
Motifs de nettoyage:  DA non clos alors que C remplacé, DA non clos alors que C déposé, DA Défaillance Mat non clos alors que C a recollecté, DA Défaillance Mat non clos alors que C coupé au CCPI
Entrez la date du jour svp ?



Jour ? (format numérique svp!)
 28


Succès !



Mois ? (format numérique svp!)
 12


Succès !



Année en cours? (format numérique svp!)
 2022


Succès !



In [5]:
date_saisie = pd.to_datetime(str(val2)+"-"+str(val1)+"-"+str(val)).strftime("%Y-%m-%d")
print("1. On cherche tous les DA non clos pour les dysfonctionnements sélectionnés (TDY_ID), et dont la création est antérieure à J-7,\n il faut donc retirer 7 jours pour l'analyse")
Q = input("Voulez retirer des jours à la date d'analyse ? oui ou non ?\n")
while Q!='oui' and Q!='non':
    print("Ecrivez correctement oui ou non en minuscule svp!\n")
    Q = input("Voulez retirer des jours à la date d'analyse ? oui ou non ?\n")
if Q=="oui":
    jour_à_retirer = input("Entrer le nombre de jour à retirer svp ?\n")
    while True:
        try:
            jour_à_retirer = int(str(jour_à_retirer))
        except ValueError:
            print("Ce n'est pas un nombre entier\n")
            jour_à_retirer = input("Entrez un nombre entier svp ?\n")
        else:
            print("Succès !\n")
            break

    date_analyse = pd.to_datetime(date_saisie) - timedelta(days=jour_à_retirer)
    date_analyse = date_analyse.strftime("%Y-%m-%d")
             
elif Q=='non':
    date_analyse = pd.to_datetime(date_saisie) - timedelta(days=0)
    date_analyse = date_analyse.strftime("%Y-%m-%d")
print("La date d'analyse est : \n",date_analyse) 
print("La date saisie est : \n",date_saisie)

1. On cherche tous les DA non clos pour les dysfonctionnements sélectionnés (TDY_ID), et dont la création est antérieure à J-7,
 il faut donc retirer 7 jours pour l'analyse


Voulez retirer des jours à la date d'analyse ? oui ou non ?
 oui
Entrer le nombre de jour à retirer svp ?
 7


Succès !

La date d'analyse est : 
 2022-12-21
La date saisie est : 
 2022-12-28


In [6]:
my_conn_sup = imlconnect.Sql("lsup_conn", 
                  key_path='/var/projects/iml/rstudio/D86553/python_conn/imlconnect.key', 
                  config_file_path='/var/projects/iml/rstudio/D86553/python_conn/config_imlconnect.yml',
                  open_connection=True)

In [7]:
# Objectif du script : identifier des dossiers d'analyse qui doivent être clos par Hypervision
# DA concernés : 30,64,25,38,23,24,39,28,42,27,54,22,61,26,59,29,56
# Motifs de nettoyage:  DA non clos alors que C remplacé, DA non clos alors que C déposé, DA Défaillance Mat non clos alors que C a recollecté, DA Défaillance Mat non clos alors que C coupé au CCPI



# 1) Chercher tous les DA non clos pour les dysfonctionnements sélectionnés (TDY_ID), et dont la création est antérieure à J-7
df_Liste_DA_C = my_conn_sup.query("select LOC_CODE DR, DAN_NUMERO_DOSSIER NUMERO_DA, TDY.TDY_LIBELLE TYPE_DA, TDY_CODE DYSF,PDC_ID PDC, PDC_ID_PRM_CONSO PRM, STC_LIBELLE STATUT_PDC, PDC_ID_C,utl_raw.cast_to_varchar2(dbms_lob.substr(DYS_SPECIFICITE)) SPECIFICITE from lspmet.t_dossier_analyse DA join lspmet.t_dysfonctionnement DYS on DYS.DYS_ID= DA.DAN_DYSFONCTIONNEMENT_FK JOIN lspmet.T_REF_TYPE_DYSFONCTIONNEMENT TDY ON DYS.DYS_TYPE_DYSFONCTIONNEMENT_FK = TDY.TDY_ID join lspmet.T_PDC on DYS_ID_OBJET=PDC_ID join lspmet.T_REF_STATUT_C on PDC_STATUT_FK=STC_ID join lspmet.t_localisation_dossier lod on lod.lod_dossier_fk = da.dan_id join lspmet.t_ref_localisation_erdf loc on loc.loc_id=lod.lod_localisation_fk WHERE TDY_ID in (30,64,25,38,23,24,39,28,42,27,54,22,61,26,59,29,56) and DAN_STATUT_DOSSIER_ANALYSE_FK=2 and dan_date_creation < to_date('"+ date_analyse + "','yyyy-mm-dd')")

In [8]:
df_Liste_DA_C = df_Liste_DA_C.assign(id_c_da ='')
for i in df_Liste_DA_C.specificite.index:
    index_debut = df_Liste_DA_C.specificite[i].find('PDC')-14
    index_fin = index_debut+12
    df_Liste_DA_C.id_c_da[i] = df_Liste_DA_C.specificite[i][index_debut:index_fin]

In [9]:
df_Liste_DA_C.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 72508 entries, 0 to 72507
Data columns (total 10 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   dr           72508 non-null  object
 1   numero_da    72508 non-null  object
 2   type_da      72508 non-null  object
 3   dysf         72508 non-null  object
 4   pdc          72508 non-null  object
 5   prm          72505 non-null  object
 6   statut_pdc   72508 non-null  object
 7   pdc_id_c     72307 non-null  object
 8   specificite  72508 non-null  object
 9   id_c_da      72508 non-null  object
dtypes: object(10)
memory usage: 5.5+ MB


In [10]:
sql_ini = "select COMPTEUR_ID from lptdwh_ods.compteur c inner join filter_list f on f.val = c.compteur_id"
sql = fc.create_sql_with_df(df_Liste_DA_C,'id_c_da',sql_ini)

In [11]:
my_conn_pilot = imlconnect.Sql("pilot_conn", 
                      key_path='/var/projects/iml/rstudio/D86553/python_conn/imlconnect.key', 
                      config_file_path='/var/projects/iml/rstudio/D86553/python_conn/config_imlconnect.yml',
                      open_connection=True)

In [12]:
df_PilotCompteur = my_conn_pilot.query(sql)

In [13]:
df_PilotCompteur.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 72099 entries, 0 to 72098
Data columns (total 1 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   compteur_id  72099 non-null  object
dtypes: object(1)
memory usage: 563.4+ KB


In [14]:
df_Liste_DA_C_1 = df_Liste_DA_C.merge(df_PilotCompteur.rename(columns={'compteur_id': 'id_c_da'}),on = 'id_c_da', how = 'inner')

In [15]:
df_Remplace_ou_Depose = df_Liste_DA_C_1[df_Liste_DA_C_1.id_c_da != df_Liste_DA_C_1.pdc_id_c]

In [16]:
df_Remplace_ou_Depose = df_Remplace_ou_Depose.assign(categorie = np.where(df_Remplace_ou_Depose.pdc_id_c.isnull()==True,'Déposé','Remplacé'))

In [17]:
df_Liste_DA_C_Prodef_NR = df_Liste_DA_C_1.query("type_da=='C - Défaillance matériel' and pdc_id_c==id_c_da").assign(categorie = 'Non Remplacé')

In [18]:
df_Liste_DA_C_DemandeInter_NR = df_Liste_DA_C_1[df_Liste_DA_C_1.type_da.isin(['C - Demande d\'intervention','C - Demande d\'intervention en masse'])]

In [19]:
df_Liste_DA_C_DemandeInter_NR = df_Liste_DA_C_DemandeInter_NR.query("pdc_id_c==id_c_da").assign(categorie = 'Non Remplacé')

In [20]:
sql_ini1 = "select DAN_NUMERO_DOSSIER NUMERO_DA,COM_COMMENTAIRE from lspmet.t_dossier_analyse da inner join filter_list f on f.val = da.dan_numero_dossier inner join lspmet.t_commentaire_da comm on da.dan_id = comm.com_da_id_fk"
sql1 = fc.create_sql_with_df(df_Liste_DA_C_DemandeInter_NR,'numero_da',sql_ini1)

In [21]:
df_Comm_DA_DemandeInter = my_conn_sup.query(sql1)

In [22]:
df_Comm_DA_DemandeInter = df_Comm_DA_DemandeInter[df_Comm_DA_DemandeInter.com_commentaire.str.contains("PRODEF|ne communique|silencieux|LTE|sans collecte|non collectant|non communicant|défaillant|C HS|C perdu|arrivée haute|bagotant|bagottant|itron ah|silen|pas de collecte|figé|tonemask|plus com|muet|compteur HS|plus de com|plus de com|pas de collecte|plus de collecte|défaillance|injoignable|Abs de com|pro def",case=False,na=False)==True].groupby(by='numero_da',as_index=False).sum()

In [23]:
df_Liste_DA_C_DemandeInter_NR = df_Liste_DA_C_DemandeInter_NR.merge(df_Comm_DA_DemandeInter.drop('com_commentaire',axis=1),on = 'numero_da', how='inner')

In [24]:
df_Liste_DA_C_Prodef_Inter_NR = pd.concat([df_Liste_DA_C_DemandeInter_NR,df_Liste_DA_C_Prodef_NR])

In [25]:
date_saisie = pd.to_datetime(str(val2)+"-"+str(val1)+"-"+str(val)).strftime("%Y-%m-%d")
print("Il faut maintenant retirer 1 jour pour la collecte de données dans pilot\n") 
Q = input("Voulez retirer des jours à la date d'analyse ? oui ou non ?\n")
while Q!='oui' and Q!='non':
    print("Ecrivez correctement oui ou non en minuscule svp!\n")
    Q = input("Voulez retirer des jours à la date d'analyse ? oui ou non ?\n")
if Q=="oui":
    jour_à_retirer = input("Entrer le nombre de jour à retirer svp ?\n")
    while True:
        try:
            jour_à_retirer = int(str(jour_à_retirer))
        except ValueError:
            print("Ce n'est pas un nombre entier\n")
            jour_à_retirer = input("Entrez un nombre entier svp ?\n")
        else:
            print("Succès !\n")
            break
   
    date_analyse1 = pd.to_datetime(date_saisie) - timedelta(days=jour_à_retirer)
    date_analyse1 = date_analyse1.strftime("%Y-%m-%d")
             
elif Q=='non':
    date_analyse1 = pd.to_datetime(date_saisie) - timedelta(days=0)
    date_analyse1 = date_analyse1.strftime("%Y-%m-%d")
print("La date d'analyse pour pilotcollecte est : \n",date_analyse1) 
print("La date saisie est : \n",date_saisie)

Il faut maintenant retirer 1 jour pour la collecte de données dans pilot



Voulez retirer des jours à la date d'analyse ? oui ou non ?
 oui
Entrer le nombre de jour à retirer svp ?
 1


Succès !

La date d'analyse pour pilotcollecte est : 
 2022-12-27
La date saisie est : 
 2022-12-28


In [26]:
sql_ini1 = "select ID_COMPTEUR,  ID_PDC, NB_JOUR_COLLECTE,NB_JOUR_NON_COLLECTE from lptdwh_dmt.fait_pdc fpdc inner join filter_list f on f.val = fpdc.id_pdc where id_jour=to_date('"+ date_analyse1 + "','yyyy-mm-dd')"
sql1 = fc.create_sql_with_df(df_Liste_DA_C_Prodef_Inter_NR,'pdc',sql_ini1)

In [27]:
df_PilotCollecte = my_conn_pilot.query(sql1)

In [28]:
df_PilotCollecte.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 49687 entries, 0 to 49686
Data columns (total 4 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   id_compteur           49687 non-null  object
 1   id_pdc                49687 non-null  object
 2   nb_jour_collecte      49687 non-null  int64 
 3   nb_jour_non_collecte  49687 non-null  int64 
dtypes: int64(2), object(2)
memory usage: 1.5+ MB


In [29]:
df_Liste_DA_C_Prodef_Inter_NR_Reprise = df_Liste_DA_C_Prodef_Inter_NR.merge(df_PilotCollecte.rename(columns={'id_pdc':'pdc'}), on = 'pdc', how = 'left').query("nb_jour_collecte>0 and nb_jour_non_collecte<=15") 

In [30]:
CCPI= pd.read_csv("/var/projects/iml/rstudio/Perf/INDUS/PROD/MeteOp/input/exclusions_ccpi_ginko.csv",sep=";",dtype=str)

In [31]:
df_Liste_DA_C_Prodef_Inter_NR_CCPI = df_Liste_DA_C_Prodef_Inter_NR.merge(CCPI.rename(columns={'ID_COMPTEUR':'id_c_da'}), on = 'id_c_da', how = 'inner')                                                                   

In [32]:
df_Nettoyage_Rempl = df_Remplace_ou_Depose[df_Remplace_ou_Depose.categorie =='Remplacé'][['numero_da','type_da','dysf','pdc','prm']].assign(cat='Remplacé')

In [33]:
df_Nettoyage_Dep = df_Remplace_ou_Depose[df_Remplace_ou_Depose.categorie =='Déposé'][['numero_da','type_da','dysf','pdc','prm']].assign(cat='Déposé')

In [34]:
df_Nettoyage_Prodef_Inter_Coll = df_Liste_DA_C_Prodef_Inter_NR_Reprise[['numero_da', 'type_da', 'dysf','pdc', 'prm']].assign(cat='Reprise Collecte')

In [35]:
df_Nettoyage_Prodef_Inter_CCPI = df_Liste_DA_C_Prodef_Inter_NR_CCPI[['numero_da', 'type_da', 'dysf', 'pdc', 'prm']].assign(cat='Coupure CCPI')

In [36]:
df_Nettoyage = pd.concat([df_Nettoyage_Rempl,df_Nettoyage_Dep,df_Nettoyage_Prodef_Inter_Coll,df_Nettoyage_Prodef_Inter_CCPI])

In [37]:
df_Nettoyage.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 599 entries, 1920 to 46
Data columns (total 6 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   numero_da  599 non-null    object
 1   type_da    599 non-null    object
 2   dysf       599 non-null    object
 3   pdc        599 non-null    object
 4   prm        599 non-null    object
 5   cat        599 non-null    object
dtypes: object(6)
memory usage: 32.8+ KB


In [38]:
df_Nettoyage_1 = df_Nettoyage.copy()
df_Nettoyage_1.type_da = np.where( (df_Nettoyage_1.dysf =='DYS_C_DEFAILLANCE_MATERIEL') & (df_Nettoyage_1.numero_da.str[:8] >'DA210613') ,'C - Défaillance matériel V2',df_Nettoyage_1.type_da)

In [39]:
Stats_Metropole = df_Nettoyage_1[~df_Nettoyage_1.prm.str[:3].isin(['757','761','762','763','764'])].assign(nb_da=1).groupby(by='cat',as_index=False).nb_da.sum()
print(Stats_Metropole)

                cat  nb_da
0      Coupure CCPI     47
1            Déposé    185
2          Remplacé    175
3  Reprise Collecte     42


In [40]:
Stats_SEI = df_Nettoyage_1[df_Nettoyage_1.prm.str[:3].isin(['757','761','762','763','764'])].assign(nb_da=1).groupby(by='cat',as_index=False).nb_da.sum()
print(Stats_SEI)

                cat  nb_da
0            Déposé      5
1          Remplacé    129
2  Reprise Collecte     16


In [41]:
Action_LSUP = df_Nettoyage_1[['numero_da', 'dysf', 'pdc']]
Action_LSUP = Action_LSUP.reset_index(drop=True)

In [42]:
import os 
directory = os.getcwd()
rep=directory+"/export_nettoyage_da/"

fc.split_df_to_csv(Action_LSUP,999,rep,date_saisie,"Action_LSUP")

# zip du dossier
import shutil as sh
sh.make_archive('export_nettoyage_da','zip',rep)
stop = time.process_time()